In [2]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter


import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier

In [5]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'

## load data, features

In [6]:
%%time
data_train = pd.read_csv(os.path.join(DATA_OWN, 'data_train.csv'), parse_dates=['timestamp'])
data_train.head()

Wall time: 1min 30s


,Unnamed: 0,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,client_freq_main_screen,...,tod_diff_glob_freq_main_screen,tod_diff_glob_freq_statement,tod_diff_glob_freq_credit_info,tod_diff_glob_freq_own_transfer,tod_diff_glob_freq_mobile_recharge,tod_diff_glob_freq_phone_money_transfer,tod_diff_glob_freq_card2card_transfer,tod_diff_glob_freq_chat,tod_diff_glob_freq_card_recharge,tod_diff_glob_freq_invest
0,0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,15,0,14,day,0.133333,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
1,1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,21,5,12,day,0.389706,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
2,2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,24,4,18,evening,0.116279,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
3,3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,15,6,19,evening,0.457831,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
4,4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,20,3,7,morning,0.600000,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184


In [7]:
(data_train.isnull().values.any())

False

In [7]:
target = data_train.multi_class_target

In [10]:
%%time
X_train, X_val, y_train, y_val = train_test_split(data_train, target, test_size=0.33, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

Wall time: 11.4 s


((3393784, 79), (1671566, 79), (3393784,), (1671566,))

In [11]:
using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))

In [43]:
#f1 = make_scorer(f1_score , average='macro')

## check classifiers with check time of work

In [ ]:
#clf_sgd_log   = SGDClassifier(loss = 'log', class_weight='balanced', n_jobs=-1)

#clf_knn  = KNeighborsClassifier()
#clf_svc  = SVC()
#clf_gaus = GaussianNB()

In [12]:
%%time
#clf_sgd_hinge = SGDClassifier(loss = 'hinge', class_weight='balanced', n_jobs=-1)
clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(X_train[using_features], y_train)
pred_sgd_hinge = clf_sgd_hinge.predict(X_val[using_features])
print(len(set(pred_sgd_hinge)), set(pred_sgd_hinge))
print(f1_score(y_val, pred_sgd_hinge, average  = 'micro'))

10 {'statement', 'chat', 'card2card_transfer', 'mobile_recharge', 'invest', 'main_screen', 'own_transfer', 'card_recharge', 'credit_info', 'phone_money_transfer'}
0.6399196920731817
Wall time: 53.8 s


In [ ]:
#%%time
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', class_weight='balanced', n_jobs=-1)
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', n_jobs=-1)
#clf_sgd_perc.fit(X_train[using_features], y_train)
#pred_sgd_perc = clf_sgd_perc.predict(X_val[using_features])
#rint(len(set(pred_sgd_perc)), set(pred_sgd_perc))
#rint(f1_score(y_val, pred_sgd_perc, average  = 'micro'))

In [ ]:
%%time
# 9 min

clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(X_train[using_features], y_train)
pred_mlp = clf_mlp.predict(X_val[using_features])
print(set(pred_mlp))
print(f1_score(y_val, pred_mlp, average  = 'micro'))

In [ ]:
#%%time
# TOOO LONG

#clf_svc.fit(X_train[using_features], y_train)
#pred_svc = clf_svc.predict(X_val[using_features])
#print(set(pred_svc))
#print(f1_score(y_val, pred_svc, average  = 'micro'))

In [ ]:
#%%time
# 24 min
#
#clf_rf   = RandomForestClassifier(n_jobs = -1, verbose = 1) 
#clf_rf.fit(X_train[using_features], y_train)
#pred_rf = clf_rf.predict(X_val[using_features])
#print(set(pred_rf))
#print(f1_score(y_val, pred_rf, average  = 'micro'))

In [ ]:
#%%time
#clf_gaus.fit(X_train[using_features], y_train)
#pred_gaus = clf_gaus.predict(X_val[using_features])
#print(set(pred_gaus))
#print(f1_score(y_val, pred_gaus, average  = 'micro'))

In [15]:
#%%time
#
#
#adaboost_inp_clf = DecisionTreeClassifier()
#clf_ab   = AdaBoostClassifier(adaboost_inp_clf)
#clf_ab.fit(X_train[using_features], y_train)
#pred_ab = clf_ab.predict(X_val[using_features])
#rint(len(set(pred_ab)), set(pred_ab))
#print(f1_score(y_val, pred_ab, average  = 'micro'))

In [13]:
%%time

clf_lr = LogisticRegression(n_jobs = -1, verbose = 1) #‘liblinear’,
clf_lr.fit(X_train[using_features], y_train)
pred_lr = clf_lr.predict(X_val[using_features])
print(len(set(pred_lr)), set(pred_lr))
print(f1_score(y_val, pred_lr, average  = 'micro'))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.3min finished


10 {'statement', 'chat', 'card2card_transfer', 'mobile_recharge', 'invest', 'main_screen', 'own_transfer', 'card_recharge', 'credit_info', 'phone_money_transfer'}
0.64655239458089
Wall time: 8min 40s


In [14]:
%%time

clf_lrsvc = LinearSVC(verbose = 1) # loss = ‘hinge’
clf_lrsvc.fit(X_train[using_features], y_train)
pred_lrsvc = clf_lrsvc.predict(X_val[using_features])
print(len(set(pred_lrsvc)), set(pred_lrsvc))
print(f1_score(y_val, pred_lrsvc, average  = 'micro'))

10 {'statement', 'chat', 'card2card_transfer', 'mobile_recharge', 'invest', 'main_screen', 'own_transfer', 'card_recharge', 'credit_info', 'phone_money_transfer'}
0.6485319754050992
Wall time: 16min 34s


In [1]:
#%%time
#
#clf_lrsvc_hinge = LinearSVC( loss = 'hinge')
#clf_lrsvc_hinge.fit(X_train[using_features], y_train)
#pred_lrsvc_hinge = clf_lrsvc_hinge.predict(X_val[using_features])
#print(len(set(pred_lrsvc_hinge)), set(pred_lrsvc_hinge))
#print(f1_score(y_val, pred_lrsvc_hinge, average  = 'micro'))

In [ ]:
#%%time
#clf_cb = CatBoostClassifier()
#clf_cb.fit(X_train[using_features], y_train)
#pred_cb = clf_cb.predict(X_val[using_features])
#print(set(pred_cb))
#print(f1_score(y_val, pred_cb, average  = 'micro'))

### saw how many give us stack

In [19]:
%%time
pred_stack = [''] * pred_sgd_hinge.shape[0]
for idx in range(pred_sgd_hinge.shape[0]):
    cnt = Counter([
                   pred_sgd_hinge[idx], 
                   #pred_mlp[idx], 
                   pred_lr[idx], 
                   pred_lrsvc[idx]
               ]).most_common()
    pred_stack[idx] = cnt[0][0]
    
    if cnt[0][0] == cnt[1][0]:
        print('terrible!')

Wall time: 8.68 s


In [21]:
print(f1_score(y_val, pred_stack, average  = 'micro'))

0.648381218569892


## classifiers GridSearch

## study best models on full data

In [23]:
%%time
clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(data_train[using_features], target)

Wall time: 59.7 s


SGDClassifier(n_jobs=-1)

In [ ]:
%%time
clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(data_train[using_features], target)

In [24]:
%%time
clf_lr = LogisticRegression(n_jobs = -1, verbose = 1)
clf_lr.fit(data_train[using_features], target)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 13.5min finished


LogisticRegression(n_jobs=-1, verbose=1)

In [25]:
%%time
clf_lrsvc = LinearSVC(verbose = True) # loss = ‘hinge’
clf_lrsvc.fit(data_train[using_features], target)

[LibLinear]Wall time: 24min 18s


LinearSVC(verbose=True)

## save models

In [26]:
%%time
pickle.dump(clf_sgd_hinge, open(os.path.join(MODELS, 'clf_sgd.pkl'), 'wb'))
#pickle.dump(clf_mlp, open(os.path.join(MODELS, 'clf_mlp.pkl'), 'wb'))
pickle.dump(clf_lr, open(os.path.join(MODELS, 'clf_lr.pkl'), 'wb'))
pickle.dump(clf_lrsvc, open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'wb'))
#pickle.dump(clf_svc, open(os.path.join(MODELS, 'clf_svc.pkl'), 'wb'))
#pickle.dump(clf_rf,  open(os.path.join(MODELS, 'clf_rf.pkl'),  'wb'))
            
#clf_cb.save_model(os.path.join(MODELS, 'clf_cb.cbm'), format='cbm')

Wall time: 501 ms
